In [13]:
import numpy as np
import mindspore
import mindspore.ops as ops
from mindspore import nn
from mindspore.dataset import vision, transforms
from mindspore.dataset import MnistDataset

from download import download

In [14]:
# url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
#       "notebook/datasets/MNIST_Data.zip"
# path = download(url, "./", kind="zip", replace=True)

In [15]:
import mindspore
print(mindspore.__file__)

c:\Users\eduardo.guimaraes\Documents\rede_neural\rede-nerural\.venv\lib\site-packages\mindspore\__init__.py


In [16]:
train_dataset = MnistDataset('MNIST_Data/train')
test_dataset = MnistDataset('MNIST_Data/test')

In [17]:
print(test_dataset.get_col_names())

['image', 'label']


In [18]:

def datapipe(dataset, batch_size):
    image_transforms = [
        vision.Rescale(1.0 / 255.0, 0),
        vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        vision.HWC2CHW()
    ]
    
    label_transform = transforms.TypeCast(mindspore.int32)

    dataset = dataset.map(image_transforms, 'image')
    dataset = dataset.map(label_transform, 'label')
    dataset = dataset.batch(batch_size)
    return dataset


In [19]:
train_dataset = datapipe(train_dataset, 64)
test_dataset = datapipe(test_dataset, 64)


In [20]:
image, label = next(train_dataset.create_tuple_iterator())
print(image.shape, image.dtype)

(64, 1, 28, 28) Float32


In [21]:
for image ,label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break


Shape of image [N, C, H, W]: (64, 1, 28, 28) Float32
Shape of label: (64,) Int32


In [22]:
for data in test_dataset.create_dict_iterator():
    print(f"Shape of image [N, C, H, W]: {data['image'].shape} {data['image'].dtype}")
    print(f"Shape of label: {data['label'].shape} {data['label'].dtype}")
    break

Shape of image [N, C, H, W]: (64, 1, 28, 28) Float32
Shape of label: (64,) Int32


In [23]:

# Define model
class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512),
            nn.ReLU(),
            nn.Dense(512, 512),
            nn.ReLU(),
            nn.Dense(512, 10)
        )

    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

model = Network()
print(model)


Network<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): ReLU<>
    (4): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >


In [24]:
epochs = 3
batch_size = 32
learning_rate = 1e-2

In [25]:
# Instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.SGD(model.trainable_params(), learning_rate)

In [26]:

def train(model, dataset, loss_fn, optimizer):
    # Define forward function
    def forward_fn(data, label):
        logits = model(data)
        loss = loss_fn(logits, label)
        return loss, logits

    # Get gradient function
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

    # Define function of one-step training
    def train_step(data, label):
        (loss, _), grads = grad_fn(data, label)
        loss = ops.depend(loss, optimizer(grads))
        return loss

    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)

        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")


In [27]:

def test(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()
    test_loss /= num_batches
    correct /= total
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [28]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataset, loss_fn, optimizer)
    test(model, test_dataset, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.302272  [  0/938]
loss: 2.289392  [100/938]
loss: 2.260535  [200/938]
loss: 2.197483  [300/938]
loss: 1.934773  [400/938]
loss: 1.410201  [500/938]
loss: 1.090883  [600/938]
loss: 0.897320  [700/938]
loss: 0.618439  [800/938]
loss: 0.503437  [900/938]
Test: 
 Accuracy: 84.3%, Avg loss: 0.533513 

Epoch 2
-------------------------------
loss: 0.479065  [  0/938]
loss: 0.423116  [100/938]
loss: 0.499911  [200/938]
loss: 0.384620  [300/938]
loss: 0.527169  [400/938]
loss: 0.380792  [500/938]
loss: 0.305747  [600/938]
loss: 0.433303  [700/938]
loss: 0.212278  [800/938]
loss: 0.285422  [900/938]
Test: 
 Accuracy: 90.1%, Avg loss: 0.338071 

Epoch 3
-------------------------------
loss: 0.405559  [  0/938]
loss: 0.286696  [100/938]
loss: 0.380016  [200/938]
loss: 0.237209  [300/938]
loss: 0.514415  [400/938]
loss: 0.340188  [500/938]
loss: 0.268519  [600/938]
loss: 0.391754  [700/938]
loss: 0.293608  [800/938]
loss: 0.153295  [900/938]
Test: 
 

In [29]:
# Save checkpoint
mindspore.save_checkpoint(model, "model.ckpt")
print("Saved Model to model.ckpt")

Saved Model to model.ckpt
